# Experimentovanie a vyhodnocovanie

Miroslav Čulík a Andrej Gáfrik

Otázky:
- MSE vs R2 - ak odstránime z input df inštancie s cenou viac ako threshold (815000), vieme znižiť MSE, RMSE ale spadne nam aj R2 - ktora metrika by mala byť prioritná
- primitívny prediktor - pred všetky inštancie budeme predikovať priemernú cenu a porovnáme MSE, RMSE, R2,...
- môžu sa odstraňovať outlieri po transformácii? nemali by sme nahrádzať hodnoty outlierov takými, akými sme ich identifikovali?
- RandomSearch pri RF: Ktorý model je lepší - ten čo má nižšiu delta_R2 alebo ten čo má najvyšší R2_test?

TODOs:
- prekopčiť veci z minulých správ do finálnej správy na overleafe https://www.overleaf.com/project/5e7101e344e1ac0001e3041b
- Experimenty
    - DM metóda: experimentovanie, nastavenie parametrov, trénovanie, testovanie, ...
    - vyhodnotenie: vyhodnotenie výsledkov, porovnanie metód (aj s publikovanými prácami)
- Zhrnutie
    - zhodnotenie toho, čo sa v rámci projektu podarilo
    - náčrt ďalšieho vhodného smerovania projektu
- Literatúra: zoznam použitej literatúry a ich citovanie v správe


- ~~pridat nove modely~~
    - ~~RF~~
    - ~~SVR~~

- ~~hyperparameter tuning~~
    - ~~manual pri RF~~
    - ~~gridsearch~~
    - ~~random search na RF a podla neho gridsearch~~
- ~~primitívny prediktor~~
- ~~refactoring~~
- feature importance(RF, Decision Tree)
- upravit pipeline pre data mensie ako threshold, ktory je r_whisker
- optional: skusit ako to bude ked dame namiesto boxcox pri cene log transformaciu


In [1]:
cd ..

C:\Users\Andrej\Desktop\OZNAL\oznal_project


In [2]:
import seaborn as sns
import random
import itertools
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.special import inv_boxcox
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from numpy.polynomial.polynomial import polyfit

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDRegressor

from src import analysis, preprocessing2, feature_selection2, metrics2, tuning3

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
path_housing = "data\\kc_house_data.csv"
df = pd.read_csv(path_housing, sep=",")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00000,3,1.00000,1180,5650,1.00000,0,0,3,7,1180,0,1955,0,98178,47.51120,-122.25700,1340,5650
1,6414100192,20141209T000000,538000.00000,3,2.25000,2570,7242,2.00000,0,0,3,7,2170,400,1951,1991,98125,47.72100,-122.31900,1690,7639
2,5631500400,20150225T000000,180000.00000,2,1.00000,770,10000,1.00000,0,0,3,6,770,0,1933,0,98028,47.73790,-122.23300,2720,8062
3,2487200875,20141209T000000,604000.00000,4,3.00000,1960,5000,1.00000,0,0,5,7,1050,910,1965,0,98136,47.52080,-122.39300,1360,5000
4,1954400510,20150218T000000,510000.00000,3,2.00000,1680,8080,1.00000,0,0,3,8,1680,0,1987,0,98074,47.61680,-122.04500,1800,7503


In [25]:
_,threshold = analysis.get_whiskers(df["price"])
# threshold = 815000
new_df = df[df["price"]<=threshold]
print(len(new_df))

20467


In [48]:
X_train, X_valid, X_test, y_train, y_valid, y_test, price_lambda = preprocessing2.run_pipeline(df)

In [49]:
X_train.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode_1,zipcode_2,zipcode_3,zipcode_4,zipcode_5,zipcode_6,zipcode_7,lat,long,sqft_living15,sqft_lot15,recon_age,price_per_sqft
12496,1982201345,2,1.00000,7.38084,4.19603,1.00000,0,0,4,7,3.41952,0.00000,1944,0,0,0,0,0,0,0,1,47.66390,-122.36400,7.04752,3.99849,70,0.00248
775,4040800810,3,2.25000,8.50943,4.29079,1.00000,0,0,4,8,3.46649,31.62278,1963,0,0,0,0,0,0,1,0,47.61880,-122.11400,7.63530,4.10521,51,0.00172
17400,7525410190,3,1.75000,8.98009,4.52123,1.50000,0,0,3,8,3.66089,0.00000,1979,0,0,0,0,0,0,1,1,47.57470,-122.03500,7.85941,4.09276,35,0.00131
17038,2326059099,4,2.50000,9.14293,4.54771,2.00000,0,0,3,9,3.68127,0.00000,2001,0,0,0,0,0,1,0,0,47.72320,-122.13100,8.11672,4.09276,13,0.00157
5043,4219400290,5,2.75000,8.98009,4.32003,1.50000,0,0,3,8,3.66089,0.00000,1939,0,0,0,0,0,1,0,1,47.65520,-122.27800,7.98616,4.06938,75,0.00169


## Experimenty

Po predspracovaní dát nami vytvorenou pipeline, sme mali dáta rozdelené v pomere 70:20:10 (trénovacia : validačná : testovacia množina). Použili sme implementácie algoritmov z modulu sklearn. Najlepšie výsledky podľa metriky R$^2$ dosiahol model polynomiálnej regresie 3. stupňa s použitím selekcie čŕt RFE (0.916 na trénovacej a 0.828 na testovacej množine), druhý najlepší výsledok podľa tejto metriky dosiahol model regresného rozhodovacieho stromu(0.861 na trénovacej a 0.802 na testovacej množine). V prípade modelu jednoduchej lineárnej regresie sme vyskúšali aj 5, resp. 10 - násobnú krížovú validáciu, na porovnanie výsledkov s modelom jednoduchej lineárnej regresie trénovaného tradičným spôsobom, avšak tento prístup nám nepreukázal žiaden signifikantný rozdiel vo výsledkoch pri modeloch jednoduchej lineárnej regresie.
 

### Primitívny prediktor
Náš primitívny prediktor bude slúžiť na porovnanie s našimi modelmi pri metrikách MSE, RMSE a RMSLE. Predikovať budeme pre každú inštanciu priemernú cenu podľa trénovacej množiny.

In [50]:
mean = np.mean(y_train)
y_hat = pd.Series([mean for _ in range(len(y_valid))])
y_hat_train = pd.Series([mean for _ in range(len(y_train))])

mse_train = metrics.mean_squared_error(inv_boxcox(y_train, price_lambda),inv_boxcox(y_hat_train, price_lambda))
rmlse_train = metrics.mean_squared_log_error(inv_boxcox(y_train, price_lambda), inv_boxcox(y_hat_train, price_lambda))

mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))

print('Mean Squared Error train', f'{round(mse_train, 5):,}')
print('Root Mean Squared Error train', f'{round(np.sqrt(mse_train),5):,}')
print('Root Mean Squared Log Error train', f'{round(np.sqrt(rmlse_train),5):,}')
print()
print('Mean Squared Error', f'{round(mean_squared_error, 5):,}')
print('Root Mean Squared Error', f'{round(np.sqrt(mean_squared_error),5):,}')
print('Root Mean Squared Log Error', f'{round(np.sqrt(rmlse),5):,}')

Mean Squared Error train 100,941,874,360.68434
Root Mean Squared Error train 317,713.50988
Root Mean Squared Log Error train 0.50631

Mean Squared Error 104,666,188,141.7946
Root Mean Squared Error 323,521.54201
Root Mean Squared Log Error 0.51184


In [29]:
np.mean(inv_boxcox(y_train, price_lambda))

476437.60269165935

### Jednoduchá lineárna regresia - sqft_living

In [51]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['sqft_living']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['sqft_living']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)

pred=reg.predict(x_test)
pred_train = reg.predict(x_train)

print('linear model - sqft_living')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, pred_train, price_lambda)

linear model - sqft_living
Mean Squared Error train 52,912,546,393.14442
Root Mean Squared Error train 230,027.27315
Root Mean Squared Log Error train 0.36918

Mean Squared Error 60,379,753,642.67178
Root Mean Squared Error 245,722.92047
Root Mean Squared Log Error 0.39115
R squared training 0.451
R squared testing 0.402
Adjusted-R squared training 0.451
Adjusted-R squared testing 0.402
intercept [3.78442207]
coefficient [[0.02965959]]


#### k-násobná krížová validácia pri jednoduchej lineárnej regresii

In [31]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train['sqft_living'], X_valid['sqft_living']])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.3747913649275906
Cross-validated scores: 0.3740271583607607


### Jednoduchá lineárna regresia - grade

In [32]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['grade']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['grade']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)

pred=reg.predict(x_test)
pred_train = reg.predict(x_train)

print('linear model - grade')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, pred_train, price_lambda)

linear model - grade
Mean Squared Error train 25,683,650,011.71324
Root Mean Squared Error train 160,261.19309
Root Mean Squared Log Error train 0.34239

Mean Squared Error 28,690,872,369.15526
Root Mean Squared Error 169,383.80197
Root Mean Squared Log Error 0.36174
R squared training 0.41
R squared testing 0.346
Adjusted-R squared training 0.41
Adjusted-R squared testing 0.346
intercept [27.78175155]
coefficient [[2.24798499]]


### Viacnásobná lineárna regresia - part1

In [34]:
features1 = feature_selection2.feature_filter(X_train, y_train, threshold=0.5)
reg=linear_model.LinearRegression()
reg.fit(X_train[features1],y_train)

pred=reg.predict(X_valid[features1])
pred_train=reg.predict(X_train[features1])

print('multiple linear regression 1')
metrics2.evaluate(reg, X_train[features1], y_train, X_valid[features1], y_valid, pred, pred_train, price_lambda)

multiple linear regression 1
Mean Squared Error train 22,275,015,653.5797
Root Mean Squared Error train 149,248.16801
Root Mean Squared Log Error train 0.31899

Mean Squared Error 25,492,063,234.17494
Root Mean Squared Error 159,662.34131
Root Mean Squared Log Error 0.34042
R squared training 0.488
R squared testing 0.42
Adjusted-R squared training 0.488
Adjusted-R squared testing 0.42
intercept [36.36455186]
coefficient [[ 0.88856235  1.42854498 -7.21184417  1.56361903]]


### Viacnásobná lineárna regresia - part2

In [35]:
features2 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)
reg=linear_model.LinearRegression()
reg.fit(X_train[features2],y_train)

pred=reg.predict(X_valid[features2])
pred_train = reg.predict(X_train[features2])

print('multiple linear regression 2')
metrics2.evaluate(reg, X_train[features2], y_train, X_valid[features2], y_valid, pred, pred_train, price_lambda)

multiple linear regression 2
Mean Squared Error train 12,608,230,125.06293
Root Mean Squared Error train 112,286.37551
Root Mean Squared Log Error train 0.22817

Mean Squared Error 15,116,743,215.97967
Root Mean Squared Error 122,950.16558
Root Mean Squared Log Error 0.24937
R squared training 0.736
R squared testing 0.686
Adjusted-R squared training 0.736
Adjusted-R squared testing 0.686
intercept [-414.51232489]
coefficient [[ 0.61122557  0.47263888  0.59441788  0.52604449  1.28146307 -0.02351915
   0.36593246 10.28908517  1.91076229 -5.20095952]]


#### k-násobná krížová validácia pri viacnásobnej lineárnej regresii

In [36]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train[features2], X_valid[features2]])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.7234398406908262
Cross-validated scores: 0.7234453567378276


### Viacnásobná lineárna regresia - part3

In [37]:
features3 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
reg=linear_model.LinearRegression()
reg.fit(X_train[features3],y_train)

pred=reg.predict(X_valid[features3])
pred_train = reg.predict(X_train[features3])

print('multiple linear regression 3')
metrics2.evaluate(reg, X_train[features3], y_train, X_valid[features3], y_valid, pred, pred_train, price_lambda)

Optimum number of features: 24 (all features: 27)
Score with 24 features: 0.692713
multiple linear regression 3
Mean Squared Error train 12,359,702,489.67468
Root Mean Squared Error train 111,174.19885
Root Mean Squared Log Error train 0.22399

Mean Squared Error 14,951,839,279.03722
Root Mean Squared Error 122,277.71375
Root Mean Squared Log Error 0.24667
R squared training 0.745
R squared testing 0.693
Adjusted-R squared training 0.744
Adjusted-R squared testing 0.691
intercept [-391.02540102]
coefficient [[-0.19373155  0.32346229  0.46483595 -1.45749151  0.33550029  2.80408969
   0.49387285  0.51812473  1.24387997  2.39437133  0.01128841 -0.02608081
  -0.17237259 -0.29776993 -0.26852878  0.04304076 -0.22111437 -0.1367936
   0.29852013 10.42974166  0.27572955  1.97476794 -3.29043902  3.21229003]]


### Polynomiálna regresia 2. stupňa 

In [38]:
features1 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)

polyfeat=PolynomialFeatures(degree=2)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])

np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)
polypred_train = poly.predict(xtrain_poly)

print('Polynomial regresion - grade 2')

mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

mse_train =metrics.mean_squared_error(inv_boxcox(np_y_train, price_lambda),inv_boxcox(polypred_train, price_lambda))
rmlse_train = metrics.mean_squared_log_error(inv_boxcox(np_y_train, price_lambda),inv_boxcox(polypred_train, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error train', f'{round(mse_train, 5):,}')
print('Root Mean Squared Error train', f'{round(np.sqrt(mse_train),5):,}')
print('Root Mean Squared Log Error train', f'{round(np.sqrt(rmlse_train),5):,}')
print()
print('Mean Squared Error', f'{round(mean_squared_error, 5):,}')
print('Root Mean Squared Error', f'{round(np.sqrt(mean_squared_error),5):,}')
print('Root Mean Squared Log Error', f'{round(np.sqrt(rmlse),5):,}')
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Polynomial regresion - grade 2
Mean Squared Error train 10,287,192,070.15601
Root Mean Squared Error train 101,425.79588
Root Mean Squared Log Error train 0.21157

Mean Squared Error 12,807,817,726.0493
Root Mean Squared Error 113,171.62951
Root Mean Squared Log Error 0.23376
R squared training 0.776
R squared testing 0.727
Adjusted-R squared training 0.775
Adjusted-R squared testing 0.727


### Polynomiálna regresia 3. stupňa

In [52]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

polyfeat=PolynomialFeatures(degree=3)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])
np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)
polypred_train = poly.predict(xtrain_poly)

print('Polynomial regresion - grade 3')
mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

mse_train =metrics.mean_squared_error(inv_boxcox(np_y_train, price_lambda),inv_boxcox(polypred_train, price_lambda))
rmlse_train = metrics.mean_squared_log_error(inv_boxcox(np_y_train, price_lambda),inv_boxcox(polypred_train, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error train', f'{round(mse_train, 5):,}')
print('Root Mean Squared Error train', f'{round(np.sqrt(mse_train),5):,}')
print('Root Mean Squared Log Error train', f'{round(np.sqrt(rmlse_train),5):,}')
print()
print('Mean Squared Error', f'{round(mean_squared_error, 5):,}')
print('Root Mean Squared Error', f'{round(np.sqrt(mean_squared_error),5):,}')
print('Root Mean Squared Log Error', f'{round(np.sqrt(rmlse),5):,}')
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
Polynomial regresion - grade 3
Mean Squared Error train 9,106,794,408.40368
Root Mean Squared Error train 95,429.52587
Root Mean Squared Log Error train 0.1432

Mean Squared Error 236,372,347,755.9625
Root Mean Squared Error 486,181.39388
Root Mean Squared Log Error 0.21942
R squared training 0.916
R squared testing 0.828
Adjusted-R squared training 0.916
Adjusted-R squared testing 0.827


In [40]:
features1

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'zipcode_1',
 'zipcode_2',
 'zipcode_3',
 'zipcode_4',
 'zipcode_5',
 'zipcode_6',
 'zipcode_7',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'price_per_sqft']

### Regresný rozhodovací strom

In [43]:
regr_1 = DecisionTreeRegressor(max_depth=10)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
y_1_train = regr_1.predict(X_train)

mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

mse_train =metrics.mean_squared_error(inv_boxcox(y_train, price_lambda),inv_boxcox(y_1_train, price_lambda))
rmlse_train = metrics.mean_squared_log_error(inv_boxcox(y_train, price_lambda),inv_boxcox(y_1_train, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error train', f'{round(mse_train, 5):,}')
print('Root Mean Squared Error train', f'{round(np.sqrt(mse_train),5):,}')
print('Root Mean Squared Log Error train', f'{round(np.sqrt(rmlse_train),5):,}')
print()
print('Mean Squared Error', f'{round(mean_squared_error, 5):,}')
print('Root Mean Squared Error', f'{round(np.sqrt(mean_squared_error),5):,}')
print('Root Mean Squared Log Error', f'{round(np.sqrt(rmlse),5):,}')
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error train 4,879,378,147.95451
Root Mean Squared Error train 69,852.54575
Root Mean Squared Log Error train 0.14601

Mean Squared Error 10,804,529,721.08483
Root Mean Squared Error 103,944.8398
Root Mean Squared Log Error 0.21522
R squared training 0.894
R squared testing 0.768
Adjusted-R squared training 0.893
Adjusted-R squared testing 0.767


### Regresný rozhodovací strom - časť 2

In [44]:
regr_1 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
y_1_train = regr_1.predict(X_train)

mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

mse_train =metrics.mean_squared_error(inv_boxcox(y_train, price_lambda),inv_boxcox(y_1_train, price_lambda))
rmlse_train = metrics.mean_squared_log_error(inv_boxcox(y_train, price_lambda),inv_boxcox(y_1_train, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error train', f'{round(mse_train, 5):,}')
print('Root Mean Squared Error train', f'{round(np.sqrt(mse_train),5):,}')
print('Root Mean Squared Log Error train', f'{round(np.sqrt(rmlse_train),5):,}')
print()
print('Mean Squared Error', f'{round(mean_squared_error, 5):,}')
print('Root Mean Squared Error', f'{round(np.sqrt(mean_squared_error),5):,}')
print('Root Mean Squared Log Error', f'{round(np.sqrt(rmlse),5):,}')
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error train 7,145,127,487.30475
Root Mean Squared Error train 84,528.85594
Root Mean Squared Log Error train 0.17573

Mean Squared Error 10,961,341,009.85199
Root Mean Squared Error 104,696.4231
Root Mean Squared Log Error 0.21467
R squared training 0.845
R squared testing 0.769
Adjusted-R squared training 0.845
Adjusted-R squared testing 0.767


### Regresný rozhodovací strom  + GridSearch

In [45]:
hyperparams = {
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'max_depth': np.arange(5,15,1),
    'max_features': np.arange(8,27,1),
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'min_impurity_split': [None],
    'min_samples_leaf': [1],
    'min_samples_split': [2],
    'min_weight_fraction_leaf': [0.0],
    'splitter': ['best']
}

trees_obj = tuning3.Trees(X_train, X_valid, y_train, y_valid, price_lambda)
df_results1 = trees_obj.grid_search(DecisionTreeRegressor, hyperparams, 50)
df_results1

Iteration 0 in process...done
Iteration 1 in process...done
Iteration 2 in process...done
Iteration 3 in process...done
Iteration 4 in process...done
Iteration 5 in process...done
Iteration 6 in process...done
Iteration 7 in process...done
Iteration 8 in process...done
Iteration 9 in process...done
Iteration 10 in process...done
Iteration 11 in process...done
Iteration 12 in process...done
Iteration 13 in process...done
Iteration 14 in process...done
Iteration 15 in process...done
Iteration 16 in process...done
Iteration 17 in process...done
Iteration 18 in process...done
Iteration 19 in process...done
Iteration 20 in process...done
Iteration 21 in process...done
Iteration 22 in process...done
Iteration 23 in process...done
Iteration 24 in process...done
Iteration 25 in process...done
Iteration 26 in process...done
Iteration 27 in process...done
Iteration 28 in process...done
Iteration 29 in process...done
Iteration 30 in process...done
Iteration 31 in process...done
Iteration 32 in pr

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params,delta_R2
0,49,11747513920.70000,108385.95000,0.22000,0.81000,0.74900,0.81000,0.74700,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.06100
1,48,11841361629.36000,108818.02000,0.22000,0.81400,0.75200,0.81400,0.75100,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.06200
2,36,12090078412.67000,109954.89000,0.22000,0.78900,0.74700,0.78800,0.74600,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04200
3,35,12166958277.67000,110303.94000,0.23000,0.78800,0.74500,0.78800,0.74400,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04300
4,46,12266953438.65000,110756.28000,0.23000,0.80500,0.74400,0.80400,0.74200,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.06100
5,32,12347644409.67000,111119.96000,0.23000,0.78600,0.74100,0.78600,0.74000,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04500
6,34,12433507554.47000,111505.64000,0.23000,0.78600,0.73900,0.78500,0.73700,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04700
7,47,12525559659.51000,111917.65000,0.23000,0.80300,0.73900,0.80200,0.73700,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.06400
8,30,12574188896.64000,112134.69000,0.23000,0.78300,0.73300,0.78300,0.73100,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.05000
9,44,12691129736.62000,112654.91000,0.23000,0.80700,0.73900,0.80700,0.73700,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.06800


### Náhodný les

In [46]:
reg_rf = RandomForestRegressor(n_estimators=100, max_depth=8)
reg_rf.fit(X_train, y_train)

y_hat = reg_rf.predict(X_valid)
y_hat_train = reg_rf.predict(X_train)

mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))
r2_train = reg_rf.score(X_train, y_train)
r2_test = reg_rf.score(X_valid, y_valid)

mse_train = metrics.mean_squared_error(inv_boxcox(y_train, price_lambda),inv_boxcox(y_hat_train, price_lambda))
rmlse_train = metrics.mean_squared_log_error(inv_boxcox(y_train, price_lambda), inv_boxcox(y_hat_train, price_lambda))

adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error train', f'{round(mse_train, 5):,}')
print('Root Mean Squared Error train', f'{round(np.sqrt(mse_train),5):,}')
print('Root Mean Squared Log Error train', f'{round(np.sqrt(rmlse_train),5):,}')
print()
print('Mean Squared Error', f'{round(mean_squared_error, 5):,}')
print('Root Mean Squared Error', f'{round(np.sqrt(mean_squared_error),5):,}')
print('Root Mean Squared Log Error', f'{round(np.sqrt(rmlse),5):,}')
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

C:\Users\Andrej\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Mean Squared Error train 5,850,210,231.08277
Root Mean Squared Error train 76,486.66702
Root Mean Squared Log Error train 0.15725

Mean Squared Error 9,076,610,061.10307
Root Mean Squared Error 95,271.24467
Root Mean Squared Log Error 0.19428
R squared training 0.876
R squared testing 0.811
Adjusted-R squared training 0.876
Adjusted-R squared testing 0.81


### Náhodný les + RandomSearch

In [57]:
hyperparams = {
#     'bootstrap': True,
#     'criterion': 'mse',
    'max_depth': np.arange(8,20,1),
    'max_features': np.arange(8,27,1),
#     'max_leaf_nodes': None,
#     'min_impurity_decrease': 0.0,
#     'min_impurity_split': None,
#     'min_samples_leaf': 1,
#     'min_samples_split': 2,
#     'min_weight_fraction_leaf': 0.0,
    'n_estimators': [10,50,200,700,1000],
#     'n_jobs': None,
#     'oob_score': False,
#     'random_state': None,
#     'verbose': 0,
#     'warm_start': False
}

df_results2 = trees_obj.random_search(RandomForestRegressor, hyperparams, 50)
df_results2

Iteration 0 in process...done
Iteration 1 in process...done
Iteration 2 in process...done
Iteration 3 in process...done
Iteration 4 in process...done
Iteration 5 in process...done
Iteration 6 in process...done
Iteration 7 in process...done
Iteration 8 in process...done
Iteration 9 in process...done
Iteration 10 in process...done
Iteration 11 in process...done
Iteration 12 in process...done
Iteration 13 in process...done
Iteration 14 in process...done
Iteration 15 in process...done
Iteration 16 in process...done
Iteration 17 in process...done
Iteration 18 in process...done
Iteration 19 in process...done
Iteration 20 in process...done
Iteration 21 in process...done
Iteration 22 in process...done
Iteration 23 in process...done
Iteration 24 in process...done
Iteration 25 in process...done
Iteration 26 in process...done
Iteration 27 in process...done
Iteration 28 in process...done
Iteration 29 in process...done
Iteration 30 in process...done
Iteration 31 in process...done
Iteration 32 in pr

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params,delta_R2
0,28,5274050942.40000,72622.66000,0.18000,0.97100,0.80700,0.97100,0.80600,"{'max_depth': 16, 'max_features': 20, 'n_estim...",0.16400
1,49,5278561088.94000,72653.71000,0.18000,0.97000,0.80700,0.97000,0.80600,"{'max_depth': 16, 'max_features': 16, 'n_estim...",0.16300
2,41,5281739595.07000,72675.58000,0.18000,0.97700,0.80700,0.97700,0.80500,"{'max_depth': 19, 'max_features': 14, 'n_estim...",0.17000
3,1,5288750036.01000,72723.79000,0.18000,0.97600,0.80600,0.97600,0.80500,"{'max_depth': 18, 'max_features': 20, 'n_estim...",0.17000
4,7,5292021218.03000,72746.28000,0.18000,0.97700,0.80600,0.97700,0.80500,"{'max_depth': 19, 'max_features': 16, 'n_estim...",0.17100
5,35,5296602519.75000,72777.76000,0.18000,0.97300,0.80600,0.97300,0.80500,"{'max_depth': 17, 'max_features': 18, 'n_estim...",0.16700
6,9,5303288568.87000,72823.68000,0.18000,0.97400,0.80600,0.97400,0.80400,"{'max_depth': 17, 'max_features': 24, 'n_estim...",0.16800
7,17,5303433143.82000,72824.67000,0.18000,0.97000,0.80600,0.97000,0.80500,"{'max_depth': 16, 'max_features': 17, 'n_estim...",0.16400
8,14,5304514220.33000,72832.10000,0.18000,0.96600,0.80600,0.96500,0.80500,"{'max_depth': 15, 'max_features': 22, 'n_estim...",0.16000
9,48,5308684252.75000,72860.72000,0.18000,0.93900,0.80800,0.93900,0.80700,"{'max_depth': 12, 'max_features': 19, 'n_estim...",0.13100


In [51]:
for i in range(10):
    print(df_results2['params'][i])

{'max_depth': 16, 'max_features': 21, 'n_estimators': 1000}
{'max_depth': 19, 'max_features': 19, 'n_estimators': 1000}
{'max_depth': 19, 'max_features': 19, 'n_estimators': 1000}
{'max_depth': 18, 'max_features': 18, 'n_estimators': 700}
{'max_depth': 13, 'max_features': 23, 'n_estimators': 700}
{'max_depth': 17, 'max_features': 24, 'n_estimators': 200}
{'max_depth': 16, 'max_features': 17, 'n_estimators': 1000}
{'max_depth': 15, 'max_features': 18, 'n_estimators': 1000}
{'max_depth': 19, 'max_features': 20, 'n_estimators': 200}
{'max_depth': 18, 'max_features': 17, 'n_estimators': 1000}


### Náhodný les + GridSearch

In [ ]:
hyperparams = {
#     'bootstrap': True,
#     'criterion': 'mse',
    'max_depth': np.arange(13,19,1),
    'max_features': np.arange(20,27,1),
#     'max_leaf_nodes': None,
#     'min_impurity_decrease': 0.0,
#     'min_impurity_split': None,
#     'min_samples_leaf': 1,
#     'min_samples_split': 2,
#     'min_weight_fraction_leaf': 0.0,
    'n_estimators': [700, 800,1000],
#     'n_jobs': None,
#     'oob_score': False,
#     'random_state': None,
#     'verbose': 0,
#     'warm_start': False
}

df_results2 = trees_obj.grid_search(RandomForestRegressor, hyperparams, 50)
df_results2

### LinearSVR

In [21]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

clf = LinearSVR(tol=1e-4, loss='squared_epsilon_insensitive')
X_my_train = X_train[features1]#.values.reshape(-1,1)
X_my_valid = X_valid[features1]#.values.reshape(-1,1)

clf.fit(X_my_train, y_train)

y_hat = clf.predict(X_my_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(X_my_train, y_train)
r2_test = clf.score(X_my_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error 46836652678.26
Root Mean Squared Error 216417.77
Root Mean Squared Log Error 0.32
R squared training 0.659
R squared testing 0.6
Adjusted-R squared training 0.659
Adjusted-R squared testing 0.598


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### RandomSearch pre LinearSVR

In [7]:
param_grid = {
#      "C": np.random.uniform(1.2, 2.0, 100),
#     "dual":[True],
#     "epsilon":np.random.uniform(0.1, 0.2, 100),
#     "fit_intercept":[True],
#     "intercept_scaling":[1],
    "loss":['epsilon_insensitive', 'squared_epsilon_insensitive'],
    "max_iter":[10000],
    "tol":np.random.uniform(1e-6, 5e-4, 100),
#     "verbose":[0]
}

svr_obj = tuning3.LinearSVR(X_train, X_valid, y_train, y_valid, price_lambda)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results = svr_obj.random_search(LinearSVR, param_grid, 50, features1)
df_results

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,0,38524914674.86000,196277.65000,0.27000,0.72700,0.70600,0.72700,0.70500,"{'loss': 'squared_epsilon_insensitive', 'max_i..."


### SGD + Nystroem

In [22]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

clf = SGDRegressor(tol=1e-3, penalty='l1', shuffle=False)
feature_map_nystroem = Nystroem(gamma=.2, random_state=1, n_components=300)

data_transformed_train = feature_map_nystroem.fit_transform(X_train[features1])
data_transformed_valid = feature_map_nystroem.transform(X_valid[features1])

clf.fit(data_transformed_train, np.ravel(y_train))
y_hat = clf.predict(data_transformed_valid)


mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(data_transformed_train, y_train)
r2_test = clf.score(data_transformed_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
Mean Squared Error 120493650338.01
Root Mean Squared Error 347121.95
Root Mean Squared Log Error 0.56
R squared training -0.221
R squared testing -0.192
Adjusted-R squared training -0.223
Adjusted-R squared testing -0.198


### SGD + RandomSearch

In [6]:
param_grid = {
    'alpha': np.random.uniform(1e-5,1e-3,100),
#     'average': [True,False],
    'early_stopping': [True],
#     'epsilon': [0.1],
#     'eta0': np.random.uniform(0.001,0.1,100),
#     'fit_intercept': [True],
    'l1_ratio': np.random.uniform(0.1,0.9,100),
    'learning_rate': ['constant', 'adaptive','invscaling'],
    'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'max_iter': [10000],
    'n_iter_no_change': [5],
    'penalty': ['l1','l2','elasticnet'],
    'power_t': np.random.uniform(0.15,0.4,100),
    'shuffle': [False],
    "tol":np.random.uniform(1e-4, 1e-2, 100),
    'validation_fraction': np.arange(0.1, 0.3, 0.05),
}
sgd_obj = tuning3.SGD(X_train, X_valid, y_train, y_valid, price_lambda)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results2 = sgd_obj.random_search(SGDRegressor, param_grid, 50, features1)
df_results2

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,0,106324648296.28000,326074.61000,0.51000,-0.00200,-0.00100,-0.02200,-0.07500,"{'alpha': 0.00045859496912911034, 'early_stopp..."


In [14]:
df_results2["params"][3]

{'alpha': 0.00010649479903447461,
 'early_stopping': True,
 'l1_ratio': 0.2730629352844155,
 'learning_rate': 'constant',
 'loss': 'huber',
 'max_iter': 10000,
 'n_iter_no_change': 5,
 'penalty': 'l2',
 'power_t': 0.1764792717720897,
 'shuffle': False,
 'tol': 0.008364349005682366,
 'validation_fraction': 0.20000000000000004}